# Crawling Data Comments

In [2]:
%pip install google-api-python-client

  Obtaining dependency information for google-api-python-client from https://files.pythonhosted.org/packages/20/65/0127a66916b5777513dba9cc05b22aab89242750b191ccc19dd8f60d5cc9/google_api_python_client-2.168.0-py3-none-any.whl.metadata
  Obtaining dependency information for httplib2<1.0.0,>=0.19.0 from https://files.pythonhosted.org/packages/a8/6c/d2fbdaaa5959339d53ba38e94c123e4e84b8fbc4b84beb0e70d7c1608486/httplib2-0.22.0-py3-none-any.whl.metadata
  Obtaining dependency information for google-auth!=2.24.0,!=2.25.0,<3.0.0,>=1.32.0 from https://files.pythonhosted.org/packages/ce/12/ad37a1ef86006d0a0117fc06a4a00bd461c775356b534b425f00dde208ea/google_auth-2.39.0-py2.py3-none-any.whl.metadata
  Obtaining dependency information for google-auth-httplib2<1.0.0,>=0.2.0 from https://files.pythonhosted.org/packages/be/8a/fe34d2f3f9470a27b01c9e76226965863f153d5fbe276f83608562e49c04/google_auth_httplib2-0.2.0-py2.py3-none-any.whl.metadata
  Obtaining dependency information for google-api-core!=2.0.

In [4]:
import os
import pandas as pd 
from googleapiclient.discovery import build
from dotenv import load_dotenv

In [5]:
load_dotenv('.env')
API_KEY = os.getenv('GOOGLE_API_KEY')

In [28]:
def video_comments(video_id, api_key):
    replies = []
    youtube = build('youtube', 'v3', developerKey = api_key)
    video_comment = youtube.commentThreads().list(part='snippet,replies', videoId=video_id).execute()
    while video_comment:
        for item in video_comment['items']:
            published = item['snippet']['topLevelComment']['snippet']['publishedAt']
            user = item['snippet']['topLevelComment']['snippet']['authorDisplayName']
            comment = item['snippet']['topLevelComment']['snippet']['textDisplay']
            like = item['snippet']['topLevelComment']['snippet']['likeCount']

            replies.append([published, user, comment, like])
            
            replycount = item['snippet']['totalReplyCount']
            if replycount>0:
                for reply in item['replies']['comments']:
                    published = reply['snippet']['publishedAt']
                    user = reply['snippet']['authorDisplayName']
                    repl = reply['snippet']['textDisplay']
                    likerepl = reply['snippet']['likeCount']
                    replies.append([[published, user, repl, likerepl]])
            
        if 'nextPageToken' in video_comment:
            video_comment = youtube.commentThreads().list(
                part = 'snippet,replies',
                pageToken = video_comment['nextPageToken'],
                videoId = video_id
            ).execute()
        else:
            break
    return replies

In [29]:
api_key = ""

# video url = https://youtu.be/gP0YCMCJnjA
video_id = 'gP0YCMCJnjA'

In [33]:
comments = video_comments(video_id, api_key)
comments

[['2025-04-28T23:41:09Z',
  '@pikangsuangsuang1816',
  'Sia2 nonton ini ngabisin Kouta gw',
  0],
 ['2025-04-28T05:15:04Z',
  '@UjangHermansyah-z9t',
  'Kalau mendengar ungkapan pak Prabowo Subianto ini begitu sayang dan perhatian nya pak Jokowi dgn pak Prabowo . Jokowi dan Prabowo benar benar tauladan  anak bangsa Indonesia . Semoga pak Prabowo dan pak Jokowi selalu sehat dan dilindungi oleh Allah SWT ..',
  0],
 ['2025-04-27T20:59:48Z',
  '@DasihSupreswi',
  'assalamualaikum warahmatullahi wa barakatuh, Alhamdulillah hirobil alamin, semoga, pemimpin 2 kita yang ada di dunia ini mari kita bersama untuk belajar cinta mencintai , yang murni',
  0],
 ['2025-04-27T15:09:47Z', '@reyluchamv', 'Lanjut part 2 yok bisa yok', 0],
 ['2025-04-27T15:02:15Z',
  '@mbahnanonano3219',
  'Sekali sj tak loyal ke AS akan di goyang sprt 98?',
  0],
 ['2025-04-27T14:27:53Z',
  '@SyamsulBahriWRDRS',
  'Amazing&#39; dg Kondisi saat ini, dimana Geopolitik sdang galau, ditambah perang dagang yg kian menggila..

In [32]:
df =  pd.DataFrame(comments, columns=['publishedAt', 'authorDisplayName', 'textDisplay', 'likeCount'])
df

,publishedAt,authorDisplayName,textDisplay,likeCount
0,2025-04-28T23:41:09Z,@pikangsuangsuang1816,Sia2 nonton ini ngabisin Kouta gw,0.0
1,2025-04-28T05:15:04Z,@UjangHermansyah-z9t,Kalau mendengar ungkapan pak Prabowo Subianto ...,0.0
2,2025-04-27T20:59:48Z,@DasihSupreswi,"assalamualaikum warahmatullahi wa barakatuh, A...",0.0
3,2025-04-27T15:09:47Z,@reyluchamv,Lanjut part 2 yok bisa yok,0.0
4,2025-04-27T15:02:15Z,@mbahnanonano3219,Sekali sj tak loyal ke AS akan di goyang sprt 98?,0.0
...,...,...,...,...
4695,"[2025-04-07T12:38:14Z, @angeliart_, TEMPO cuma...",None,None,NaN
4696,"[2025-04-07T12:40:48Z, @sanileonardo444, @@ang...",None,None,NaN
4697,2025-04-07T12:03:05Z,@sanileonardo444,Kedua,0.0
4698,2025-04-07T12:02:56Z,@NusibMaruain,seru nih,0.0


In [34]:
df.to_csv('commentPresidenPrabowoMenjawab.csv', index=False)

# Cleaning Data Comments

In [37]:
df[df['textDisplay'].isnull()]

,publishedAt,authorDisplayName,textDisplay,likeCount
98,"[2025-04-14T17:05:44Z, @phyden06, 😂😂😂😂😂, 0]",None,None,NaN
104,"[2025-04-15T07:49:56Z, @jaarchanel3920, Tapi a...",None,None,NaN
162,"[2025-04-13T01:35:44Z, @saladinwarrior, Gibran...",None,None,NaN
174,"[2025-04-13T01:36:34Z, @saladinwarrior, harusn...",None,None,NaN
175,"[2025-04-13T01:40:00Z, @azzahrasyamilarahmat50...",None,None,NaN
...,...,...,...,...
4692,"[2025-04-07T12:14:31Z, @Andi.Maulana, Tempo ka...",None,None,NaN
4693,"[2025-04-07T12:15:49Z, @m.ichsankhotimi1033, y...",None,None,NaN
4694,"[2025-04-07T12:29:57Z, @rfach7401, ​@manusia__...",None,None,NaN
4695,"[2025-04-07T12:38:14Z, @angeliart_, TEMPO cuma...",None,None,NaN


In [38]:
def split_list_to_columns(row):
    if pd.isna(row['textDisplay']) and isinstance(row['publishedAt'], list):
        row['publishedAt'], row['authorDisplayName'], row['textDisplay'], row['likeCount'] = row['publishedAt']
    return row

In [39]:
df = df.apply(split_list_to_columns, axis=1)
df

,publishedAt,authorDisplayName,textDisplay,likeCount
0,2025-04-28T23:41:09Z,@pikangsuangsuang1816,Sia2 nonton ini ngabisin Kouta gw,0.0
1,2025-04-28T05:15:04Z,@UjangHermansyah-z9t,Kalau mendengar ungkapan pak Prabowo Subianto ...,0.0
2,2025-04-27T20:59:48Z,@DasihSupreswi,"assalamualaikum warahmatullahi wa barakatuh, A...",0.0
3,2025-04-27T15:09:47Z,@reyluchamv,Lanjut part 2 yok bisa yok,0.0
4,2025-04-27T15:02:15Z,@mbahnanonano3219,Sekali sj tak loyal ke AS akan di goyang sprt 98?,0.0
...,...,...,...,...
4695,2025-04-07T12:38:14Z,@angeliart_,TEMPO cuma berani ghibah. Giliran Diundang lan...,1.0
4696,2025-04-07T12:40:48Z,@sanileonardo444,@@angeliart_ heheh yakinnn? Eh kepala babi apa...,2.0
4697,2025-04-07T12:03:05Z,@sanileonardo444,Kedua,0.0
4698,2025-04-07T12:02:56Z,@NusibMaruain,seru nih,0.0


In [40]:
df.isnull().sum()

publishedAt          0
authorDisplayName    0
textDisplay          0
likeCount            0
dtype: int64

In [41]:
df.to_csv('comments_clean.csv', index=False)